# Imports

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path
from typing import List, Dict, Tuple
from tqdm import tqdm

In [1]:
# Uncomment these lines to install any necessary packages:

# !pip install pandas
# !pip install seaborn
# !pip install matplotlib

# Configurable Variables

In [1]:
# Dictionary mapping TF timestamp to model type
model_mapper: Dict[str, str] = {
    'run-Jul23_10-48-06': 'pretrained',
    'run-Jul23_19-36-20': 'modified',
    'run-Jul24_04-04-46': 'stylized',
    'run-Jul25_22-16-42': 'coco',
}

# Path containing CSV files from tensorboard
path_to_tf_runs = Path("linear_classifier_runs/")

# Helper Function

In [1]:
def get_last_n_values(df: pd.DataFrame, column: str, n: int = 10):
    
    for i, value in enumerate(df[column].sort_index(ascending=False)):
        yield value
        
        if i >= n:
            break

# Get last N metrics from each run

In [1]:
# List to keep track of max metrics
max_metrics: List[Tuple[str, int, float]] = []

# Iterate over directory
all_csv_files = [f for f in path_to_tf_runs.iterdir() if f.is_file()]
for csv_file in tqdm(all_csv_files):
    
    # Check if file first
    if not csv_file.is_file():
        continue
    
    # Parse layer number from file name
    layer_number: int = int(csv_file.stem.split('_')[-1][-1])
        
    # Parse model type from file name, and convert via "model_mapper" dictionary above
    model_tf_ts: str = '_'.join(csv_file.name.split('_')[:2])
    model_type: str = model_mapper[model_tf_ts]
        
    # Load csv into pandas dataframe and get best validation accuracy
    df = pd.read_csv(str(csv_file))
    
    # max_acc = df['Value'].max()
    for value in get_last_n_values(df=df, column="Value", n=3):
        max_metrics.append((model_type, layer_number, value))

df_metrics = pd.DataFrame(max_metrics, columns=['Model', 'Layer_Number', 'Max_Validation_Accuracy'])

# Create the plot

In [1]:
with sns.axes_style("darkgrid"):
    
    fig, ax = plt.subplots()
    # fig.set_size_inches(11.7, 8.27)
    fig.set_size_inches(8, 6.5)

    g = sns.lineplot(data=df_metrics, x="Layer_Number", y="Max_Validation_Accuracy", hue="Model", ax=ax, ci=95, marker="o")
    
    ax.set_title("ILSVRC2012 Linear Classification")
    ax.set_xlabel("Layer Number")
    ax.set_ylabel("Validation Accuracy")
    
    g.set_xticks(range(1, 6))
    g.set_xticklabels(['conv1','conv2','conv3','conv4','conv5'])
        